In [1]:
import re    # for regular expressions 
import nltk  # for text manipulation 
from nltk.corpus import stopwords
import string 
import warnings 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt  
import xgboost as xgb
from sklearn import tree
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
xtrain = pd.read_csv('train.csv')
xtrain

,target,long,equal_words,Tags,with_url,#words,#hashtags,text,unique_words
0,1,69,0,0,0,13,1,Our Deeds are the Reason of this #earthquake M...,13
1,1,38,0,0,0,7,0,Forest fire near La Ronge Sask. Canada,7
2,1,133,0,0,0,22,0,All residents asked to 'shelter in place' are ...,20
3,1,65,0,0,0,8,1,"13,000 people receive #wildfires evacuation or...",8
4,1,88,0,0,0,16,2,Just got sent this photo from Ruby #Alaska as ...,15
...,...,...,...,...,...,...,...,...,...
7608,1,83,0,0,1,11,0,Two giant cranes holding a bridge collapse int...,11
7609,1,125,0,2,0,20,0,@aria_ahrary @TheTawniest The out of control w...,16
7610,1,65,0,0,1,8,0,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,8
7611,1,137,0,0,0,19,0,Police investigating after an e-bike collided ...,18


In [4]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [5]:
xtrain['text'] = xtrain['text'].apply(lambda x: clean_text(x))

In [6]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
xtrain['text'] = xtrain['text'].apply(lambda x: tokenizer.tokenize(x))

In [7]:
def remove_stopwords(text):
    words = []
    for w in text:
        if w not in stopwords.words('english'):
            words.append(w)
    #words = [w for w in text if w not in stopwords.words('english')]
    return words

xtrain['text'] = xtrain['text'].apply(lambda x: remove_stopwords(x))

In [8]:
def combine_text(list_of_text):
    combined_text = ' '.join(list_of_text)
    return combined_text

In [9]:
xtrain['text'] = xtrain['text'].apply(lambda x: combine_text(x))

In [10]:
tokenized_tweet = xtrain.text.apply(lambda x: x.split())
tokenized_tweet[1]

['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada']

In [11]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_glovec[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [12]:
model_glovec = {}
with open("glove.twitter.27B.200d.txt", 'r', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        model_glovec[word] = vector

In [13]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 200)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 200)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(7613, 200)

In [14]:
xtest = pd.read_csv('test.csv')
xtest.text = xtest.text.apply(lambda x: clean_text(x))
xtest.text = xtest.text.apply(lambda x: tokenizer.tokenize(x))
xtest.text = xtest.text.apply(lambda x: remove_stopwords(x))
xtest.text = xtest.text.apply(lambda x: combine_text(x))
tokenized_test = xtest.text.apply(lambda x: x.split()) # tokenizing 
wordvec_test = np.zeros((len(tokenized_test), 200)) 
for i in range(len(tokenized_test)):
    wordvec_test[i,:] = word_vector(tokenized_test[i], 200)
wordvect_df = pd.DataFrame(wordvec_test)
wordvect_df.shape

(3263, 200)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
xtrain_gv = wordvec_df
xtest_gv = wordvect_df
ytrain= xtrain.target

In [19]:
xtrain_gv = np.asarray(xtrain_gv).reshape(-1,1,200)
xtest_gv = np.asarray(xtest_gv).reshape(-1,1,200)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(xtrain_gv, 
                                                    ytrain, 
                                                    test_size=0.25, random_state=42, 
                                                    stratify=ytrain
                                                     )

In [21]:
len(X_train)

5709

In [51]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Activation, Bidirectional, Embedding, GRU,RNN

model = Sequential()
model.add(Bidirectional(LSTM(200,input_shape=(1,207), return_sequences=True, recurrent_dropout=0.1), merge_mode="mul"))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(300,input_shape=(1,207), return_sequences=True,recurrent_dropout=0.1),merge_mode="ave"))
model.add(Dropout(0.5))
model.add(Dense(150))
model.add((LSTM(100,input_shape=(1,207), return_sequences=True,recurrent_dropout=0.1)))
model.add(GlobalMaxPooling1D())
model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.6))
model.add(Dense(450))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.build(input_shape=(6090,1,200))
model.summary()
opt = keras.optimizers.Adam(learning_rate=0.00005)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_10 (Bidirectio (6090, 1, 150)            429600    
_________________________________________________________________
conv1d_3 (Conv1D)            (6090, 1, 70)             10570     
_________________________________________________________________
dropout_7 (Dropout)          (6090, 1, 70)             0         
_________________________________________________________________
bidirectional_11 (Bidirectio (6090, 1, 70)             78960     
_________________________________________________________________
global_max_pooling1d_5 (Glob (6090, 70)                0         
_________________________________________________________________
dense_15 (Dense)             (6090, 120)               8520      
_________________________________________________________________
flatten_4 (Flatten)          (6090, 120)              

In [23]:
max_review_length = 200 
x_train = np.asarray(xtrain_gv)
from keras.preprocessing.sequence import pad_sequences
#X_train = pad_sequences(x_train, maxlen=max_review_length)
x_train


array([[[-1.22043065e-03,  2.44429858e-01, -7.32538583e-02, ...,
          1.30000000e+01,  1.00000000e+00,  1.30000000e+01]],

       [[-2.20747002e-01,  2.34415733e-02, -8.55718340e-04, ...,
          7.00000000e+00,  0.00000000e+00,  7.00000000e+00]],

       [[-2.78436369e-02,  1.26615633e-01, -3.20697912e-01, ...,
          2.20000000e+01,  0.00000000e+00,  2.00000000e+01]],

       ...,

       [[-1.03846498e-01, -1.79474987e-01,  6.76399991e-02, ...,
          8.00000000e+00,  0.00000000e+00,  8.00000000e+00]],

       [[-1.01647925e-01, -1.73949861e-01, -2.21665574e-01, ...,
          1.90000000e+01,  0.00000000e+00,  1.80000000e+01]],

       [[-1.66893874e-01,  7.66258766e-02, -3.97220122e-01, ...,
          1.30000000e+01,  0.00000000e+00,  1.30000000e+01]]])

In [24]:

Y_train = np.asarray(ytrain)
Y_train.mean()


0.4296597924602653

In [48]:
#Test training
model.fit(X_train, y_train, validation_split=0.25, epochs =40, batch_size=80)

Epoch 1/40
54/54 [==============================] - 2s 44ms/step - loss: 0.6902 - accuracy: 0.5587 - val_loss: 0.6869 - val_accuracy: 0.5700
Epoch 2/40
54/54 [==============================] - 1s 23ms/step - loss: 0.6846 - accuracy: 0.5704 - val_loss: 0.6810 - val_accuracy: 0.5700
Epoch 3/40
54/54 [==============================] - 1s 24ms/step - loss: 0.6788 - accuracy: 0.5704 - val_loss: 0.6728 - val_accuracy: 0.5700
Epoch 4/40
54/54 [==============================] - 1s 24ms/step - loss: 0.6690 - accuracy: 0.5711 - val_loss: 0.6586 - val_accuracy: 0.5707
Epoch 5/40
54/54 [==============================] - 1s 23ms/step - loss: 0.6513 - accuracy: 0.5966 - val_loss: 0.6325 - val_accuracy: 0.6303
Epoch 6/40
54/54 [==============================] - 1s 23ms/step - loss: 0.6249 - accuracy: 0.6692 - val_loss: 0.5962 - val_accuracy: 0.6961
Epoch 7/40
54/54 [==============================] - 1s 23ms/step - loss: 0.5951 - accuracy: 0.6975 - val_loss: 0.5657 - val_accuracy: 0.7136
Epoch 8/40
54

In [52]:
#Entrenamiento completo
x_train = np.asarray(xtrain_gv)
model.fit(x_train, ytrain, validation_split=0.25, epochs =35, batch_size=50)

Epoch 1/35
115/115 [==============================] - 4s 32ms/step - loss: 0.6885 - accuracy: 0.5696 - val_loss: 0.6876 - val_accuracy: 0.5494
Epoch 2/35
115/115 [==============================] - 2s 20ms/step - loss: 0.6798 - accuracy: 0.5773 - val_loss: 0.6795 - val_accuracy: 0.5494
Epoch 3/35
115/115 [==============================] - 2s 21ms/step - loss: 0.6623 - accuracy: 0.5775 - val_loss: 0.6437 - val_accuracy: 0.5494
Epoch 4/35
115/115 [==============================] - 2s 20ms/step - loss: 0.6162 - accuracy: 0.6537 - val_loss: 0.5734 - val_accuracy: 0.7306
Epoch 5/35
115/115 [==============================] - 2s 20ms/step - loss: 0.5708 - accuracy: 0.7145 - val_loss: 0.5317 - val_accuracy: 0.7489
Epoch 6/35
115/115 [==============================] - 2s 21ms/step - loss: 0.5386 - accuracy: 0.7446 - val_loss: 0.4969 - val_accuracy: 0.7673
Epoch 7/35
115/115 [==============================] - 2s 21ms/step - loss: 0.5066 - accuracy: 0.7705 - val_loss: 0.4661 - val_accuracy: 0.7826

In [49]:
#Hago tests internos
tests=model.predict(X_test)
type(tests)

numpy.ndarray

In [ ]:
y_test

In [27]:
#y_test = pd.DataFrame(y_test)
#y_test=y_test.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test.reset_index())

In [28]:
y_test.drop(columns='index', inplace=True)

In [50]:
#score_df=pd.DataFrame(columns=['test', 'target'])
score_data= pd.DataFrame(tests)
score_data.rename(columns={0:'test'}, inplace=True)
score_data.test=score_data.test.apply(lambda x: int(round(x)))
score_data['target']= y_test['target']
score_data['diff']=score_data[['test', 'target']].apply(lambda x: 1 if int((x.test).round()) == x.target else 0, axis=1)
score_data.mean()

test      0.373424
target    0.429622
diff      0.816702
dtype: float64

In [53]:
A=model.predict(xtest_gv)

In [1]:
ids=[]
with open("sample_submission.csv", 'r') as f:
    for line in f:
        vals = line.split(',')
        if vals[0]!='id':
            ids.append(int(vals[0]))


In [58]:
en = open("submission.csv", 'w')
n=0
en.write("id,target\n")
for i in ids:
    en.write(str(i) +','+ str(int((A[n].round())))+'\n')
    
    n+=1
en.close()

In [ ]:
dfrnn=pd.read_csv("submissiona.csv")
dfrnn['xgb']=A
dfrnn['diffe']=abs(dfrnn['xgb']-dfrnn['target'])


In [ ]:
dfrnn.diffe.apply('sum')